In [1]:
# Import necessary libraries
import os
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from sklearn.model_selection import train_test_split

In [ ]:
from tensorflow import keras
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import numpy as np
import cv2
import pandas as pd

# Load train data
train_data = pd.read_csv("/kaggle/input/bttai-nybg-2024/BTTAIxNYBG-train.csv")[:1000]

# Load and preprocess train images
x_train = []
for index, row in train_data.iterrows():
    image_path = "/kaggle/input/bttai-nybg-2024/BTTAIxNYBG-train/BTTAIxNYBG-train/" + row['imageFile']  # adjust the path as needed
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert to RGB
    image = cv2.resize(image, (32, 32))  # Resize to 32x32
    x_train.append(image)

x_train = np.array(x_train)
y_train = train_data['classID'].values

# Normalize train data
x_train = x_train / 255.0
# Print the shape of the train dataset
print("Train Data Shape:", x_train.shape)

# Load validation data
validation_data = pd.read_csv("/kaggle/input/bttai-nybg-2024/BTTAIxNYBG-validation.csv")[:1000]

# Load and preprocess validation images
x_validation = []
for index, row in validation_data.iterrows():
    image_path = "/kaggle/input/bttai-nybg-2024/BTTAIxNYBG-validation/BTTAIxNYBG-validation/" + row['imageFile']  # adjust the path as needed
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert to RGB
    image = cv2.resize(image, (32, 32))  # Resize to 32x32
    x_validation.append(image)

x_validation = np.array(x_validation)
y_validation = validation_data['classID'].values

# Normalize validation data
x_validation = x_validation / 255.0

# Print the shape of the validation dataset
print("Validation Data Shape:", x_validation.shape)

In [ ]:
import tensorflow as tf
from tensorflow.keras import datasets
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

#(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

#train_images, test_images = train_images / 255.0, test_images / 255.0

model = tf.keras.models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10))

model.compile(optimizer='adam', 
             loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
             metrics=['accuracy'])
#history = model.fit(train_images, train_labels, epochs=10, validation_data=(test_images, test_labels))
history = model.fit(x_train, y_train, epochs=10, validation_data=(x_validation, y_validation))

In [ ]:
# Load test data
test_data = pd.read_csv("/kaggle/input/bttai-nybg-2024/BTTAIxNYBG-test.csv")
print(test_data.shape)
print(len(test_data))
# Load and preprocess test images
x_test = []
count = 0
for index, row in test_data.iterrows():
    image_path = "/kaggle/input/bttai-nybg-2024/BTTAIxNYBG-test/BTTAIxNYBG-test/" + row['imageFile']  # adjust the path as needed
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert to RGB
    image = cv2.resize(image, (32, 32))  # Resize to 32x32
    x_test.append(image)
    count += 1
    if count % 1000 == 0:
        print(count)

x_test = np.array(x_test)

# Normalize test data
x_test = x_test / 255.0

# Print the shape of the test dataset
print("Test Data Shape:", x_test.shape)

In [ ]:
predictions = model.predict(x_test)  # Replace with your actual input data
class_predictions = np.argmax(predictions, axis=-1)
print(class_predictions)

In [ ]:
output = pd.DataFrame({'uniqueID': test_data['uniqueID'], 'ClassID': class_predictions})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

## Testing

In [2]:
# Load dataset & Define image directory
# df = pd.read_csv('/path/to/your/dataset.csv')  
# NOTE: if we have seperate train dataset and validate dataset, you may want to seperately load it with pd.read_csv() 
# Please update the paths when running this code
train_df = pd.read_csv('/kaggle/input/bttai-nybg-2024/BTTAIxNYBG-train.csv')
validate_df = pd.read_csv('/kaggle/input/bttai-nybg-2024/BTTAIxNYBG-validation.csv')

image_directory_train = '/kaggle/input/bttai-nybg-2024/BTTAIxNYBG-train'  # Please update this path when running this code
image_directory_validation = '/kaggle/input/bttai-nybg-2024/BTTxNYBG-validation'

# Preprocessing function to load and process images
def load_and_preprocess_image_train(filename, target_size=(224, 224)):
    img_path = os.path.join(image_directory_train, filename)
    img = image.load_img(img_path, target_size=target_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Model expects a batch of images
    return img_array / 255.0  # Normalize to [0, 1]

# Preprocessing function to load and process images
def load_and_preprocess_image_train(filename, target_size=(224, 224)):
    img_path = os.path.join(image_directory_validation, filename)
    img = image.load_img(img_path, target_size=target_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Model expects a batch of images
    return img_array / 255.0  # Normalize to [0, 1]

# Apply preprocessing to all images
train_df['imageData'] = train_df['imageFile'].apply(load_and_preprocess_image())
validate_df['imageData'] = validate_df['imageFile'].apply(load_and_preprocess_image())

# Split dataset into training and validation sets
### Note: This is a common step in ML training, but in this challenge, since the validation set is provided separately, there is no need to call this function to distinguish between validation and train set.
# train_df, validate_df = train_test_split(df, test_size=0.2, random_state=42)

# Data augmentation configuration for training
train_datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Note: No augmentation for validation data, only rescaling
validation_datagen = ImageDataGenerator(rescale=1./255)

# Convert dataframe to a format suitable for the model training
def df_to_dataset(dataframe, datagen, batch_size=32):
    datagen.flow_from_dataframe(
        dataframe=dataframe,
        directory=image_directory,
        x_col='imageFile',
        y_col='classLabel',
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='categorical'  # Change this if not a multiclass classification
    )

# Create datasets for training and validation
train_dataset = df_to_dataset(train_df, train_datagen)
validation_dataset = df_to_dataset(validate_df, validation_datagen)

# This setup is now ready for training with model.fit using the train_dataset and validation_dataset

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/bttai-nybg-2024/BTTAIxNYBG-train.csv'